In [41]:
import pandas as pd
from hdbcli import dbapi
import datetime
import time
import time ; from datetime import date, timedelta #biblioteca para trabalhar com tempo, datas...
import folium
from folium.plugins import Search
from folium.plugins import MarkerCluster
import folium.plugins as plugins
import numpy as np

In [33]:
usuario_hana = "CLB344173"
senha_hana = "Jun2022$"
connection_hana = dbapi.connect(address='BRNEO695',port='30015',user=usuario_hana,password=senha_hana,databasename='BNP',sslValidateCertificate=False)

In [34]:
tempo_inicial = time.time()
sql0 = """
SELECT ZCGNMLOGR,ZCGBAIRR,MUNICIPIO,"cluster" AS PACOTE,BASE.ZCGACCOUN,PC.ZCGLATITU as LATITUDE,PC.ZCGLONGIT as LONGITUDE,UTD,ZCGMTVCOB AS MONTANTE,PECLD,CARTEIRA,TURMA,DATA_PEDIDO,ZONA,NOTA,TIPO_NOTA,
RANK() OVER ( PARTITION BY UTD ORDER BY PECLD DESC ) AS RANK_PECLD,
RIGHT("cluster",1) AS ATALHO_PACOTE,
RIGHT("cluster",2) AS ATALHO_PACOTE_2
FROM (
SELECT 
	AH."cluster", 
	AH.ZCGACCOUN,
	ZCGUTD AS UTD,
	ZCGMTVCOB,
	ZCGPECLD AS PECLD,
	CASE WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJUNTOR MIX' THEN 'DISJUNTOR' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' END AS CARTEIRA,
	TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF,
	NOTA,
	TIPO_NOTA
FROM CLB961920.ATENA_HIST AH
LEFT JOIN (SELECT TAB004.ZCGQMNUM AS NOTA,
	TAB004.ZCGDTCRIA AS ZCGDTCRIA,
	TAB004.ZCGINSTAL AS INSTALACAO,
	TAB004.ZCGPEDIDO AS PEDIDO,
	TAB004.ZCGACCOUN,
	TAB004.ZCGTPNOTA AS TIPO_NOTA
FROM CLB_CCS_ICC.ZCT_DS_TAB004 TAB004
WHERE ZCGTPNOTA IN ('CA','CS','CB')) TAB004
ON TAB004.ZCGACCOUN = AH.ZCGACCOUN AND 
(ZCGDTCRIA >= ADD_DAYS(DATA_PEDIDO, 0) AND ZCGDTCRIA <= ADD_DAYS(DATA_PEDIDO, 1) )
WHERE DATA_PEDIDO = '2022-09-08'
AND FLAG_DEF = 'N'

UNION 

SELECT 
	CH."cluster",
	CH.ZCGACCOUN,
	PC.UTD,
	ZCGMTVCOB,
	NULL AS PECLD,
	CASE WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'BAIXA' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'RECORTE'
	WHEN CARTEIRA = 'DISJUNTOR' OR CARTEIRA = 'DISJ MIX' THEN 'DISJUNTOR'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'CORTE'
	WHEN (CARTEIRA = 'TOP5' OR CARTEIRA = 'TOP_STC')AND DAYOFWEEK(DATA_PEDIDO) = 4 THEN 'RECORTE'
	WHEN (CARTEIRA = 'TOP5' OR CARTEIRA = 'TOP_STC') AND DAYOFWEEK(DATA_PEDIDO) <> 4 THEN 'CORTE'
	WHEN (CARTEIRA = 'PROPRIA') AND DAYOFWEEK(DATA_PEDIDO) = 4 THEN 'RECORTE'
	WHEN (CARTEIRA = 'PROPRIA') AND DAYOFWEEK(DATA_PEDIDO) <> 4 THEN 'CORTE' END AS CARTEIRA,
	
	CASE WHEN CARTEIRA = 'BAIXA MIX' OR CARTEIRA = 'BAIXA' THEN 'EPS' 
	WHEN CARTEIRA = 'RECORTE MIX' OR CARTEIRA = 'RECORTE' THEN 'EPS'
	WHEN CARTEIRA = 'DISJ MIX' OR CARTEIRA = 'DISJUNTOR' THEN 'EPS'
	WHEN CARTEIRA = 'CONVENCIONAL' THEN 'EPS'
	WHEN (CARTEIRA = 'TOP5' ) THEN 'EPS'
	WHEN ( CARTEIRA = 'TOP_STC')  THEN 'STC'
	WHEN (CARTEIRA = 'PROPRIA') THEN 'STC' END AS TURMA,
	DATA_PEDIDO,
	DAYOFWEEK(DATA_PEDIDO) AS DIA_SEMANA,
	ZCGIDZONA AS ZONA,
	FLAG_DEF,
	NOTA,
	TIPO_NOTA
FROM CLB142840.CLUSTER_HIST CH
LEFT JOIN (SELECT TAB004.ZCGQMNUM AS NOTA,
	TAB004.ZCGDTCRIA AS ZCGDTCRIA,
	TAB004.ZCGINSTAL AS INSTALACAO,
	TAB004.ZCGPEDIDO AS PEDIDO,
	TAB004.ZCGACCOUN,
	TAB004.ZCGTPNOTA AS TIPO_NOTA
FROM CLB_CCS_ICC.ZCT_DS_TAB004 TAB004
WHERE ZCGTPNOTA IN ('CA','CS','CB')) TAB004
ON TAB004.ZCGACCOUN = CH.ZCGACCOUN AND 
(ZCGDTCRIA >= ADD_DAYS(DATA_PEDIDO, 0) AND ZCGDTCRIA <= ADD_DAYS(DATA_PEDIDO, 1) )

LEFT JOIN (SELECT DISTINCT UTD40 AS UTD, ZCGACCOUN FROM CLP123432.PLANILHAO_CLIENTE PC WHERE PC.SEQCC = 1) AS PC
ON PC.ZCGACCOUN = CH.ZCGACCOUN


WHERE CARTEIRA <> 'PRONTIDAO'
AND CH.ZCGACCOUN NOT IN (SELECT DISTINCT AH.ZCGACCOUN FROM CLB961920.ATENA_HIST AH WHERE DATA_PEDIDO = '2022-09-08')
AND DATA_PEDIDO = '2022-09-08'
AND FLAG_DEF = 'N'

) AS BASE
LEFT JOIN (
    SELECT 
        ZCGACCOUN,ZCGLATITU,ZCGLONGIT,MUNICIPIO
    FROM CLP123432.PLANILHAO_CLIENTE AS PC
    WHERE 
        SEQCC = 1
        AND ZCGLATITU IS NOT NULL
    ) PC 
ON PC.ZCGACCOUN = BASE.ZCGACCOUN

LEFT JOIN ( SELECT ZCGACCOUN,ZCGNMLOGR,ZCGBAIRR
FROM CLB_CCS_ICC.ZCT_DS_CLI001 CLI001 ) AS CLI001
ON CLI001.ZCGACCOUN = BASE.ZCGACCOUN

WHERE NOTA IS NOT NULL
    """
base_df = pd.read_sql(sql0,connection_hana)



In [35]:
base_df['LATITUDE'] = base_df['LATITUDE'].str.replace(',','.')
base_df['LONGITUDE'] = base_df['LONGITUDE'].str.replace(',','.')
base_df = base_df.dropna(subset = ['LATITUDE']).reset_index(drop = True)

gp1 = base_df[(base_df['UTD'] == "JUAZEIRO")].copy()
gp2 = base_df[(base_df['UTD'] == "JACOBINA")].copy()
gp3 = base_df[(base_df['UTD'] == "REMANSO")].copy()
gp4 = base_df[(base_df['UTD'] == "SENHOR DO BONFIM")].copy()
gp5 = base_df[(base_df['UTD'] == "ITAPETINGA")].copy()
gp6 = base_df[(base_df['UTD'] == "VITÓRIA DA CONQUISTA")].copy()
gp7 = base_df[(base_df['UTD'] == "JEQUIÉ")].copy()
gp8 = base_df[(base_df['UTD'] == "EUNÁPOLIS")].copy()
gp9 = base_df[(base_df['UTD'] == "ILHEUS")].copy()
gp10 = base_df[(base_df['UTD'] == "ITABUNA")].copy()
gp11 = base_df[(base_df['UTD'] == "PORTO SEGURO")].copy()
gp12 = base_df[(base_df['UTD'] == "POSTO DA MATA")].copy()
gp13 = base_df[(base_df['UTD'] == "TEIXEIRA DE FREITAS")].copy()
gp14 = base_df[(base_df['UTD'] == "AMARGOSA")].copy()
gp15 = base_df[(base_df['UTD'] == "IPIAU")].copy()
gp16 = base_df[(base_df['UTD'] == "SANTO ANTÔNIO DE JESUS")].copy()
gp17 = base_df[(base_df['UTD'] == "VALENÇA")].copy()
gp18 = base_df[(base_df['UTD'] == "RIBEIRA DO POMBAL")].copy()
gp19 = base_df[(base_df['UTD'] == "ALAGOINHAS")].copy()
gp20 = base_df[(base_df['UTD'] == "ESPLANADA")].copy()
gp21 = base_df[(base_df['UTD'] == "PAULO AFONSO")].copy()
gp22 = base_df[(base_df['UTD'] == "FEIRA DE SANTANA SUL")].copy()
gp23 = base_df[(base_df['UTD'] == "FEIRA DE SANTANA NORTE")].copy()
gp24 = base_df[(base_df['UTD'] == "SERRINHA")].copy()
gp25 = base_df[(base_df['UTD'] == "CONCEICAO DO COITE")].copy()
gp26 = base_df[(base_df['UTD'] == "SANTO AMARO")].copy()
gp27 = base_df[(base_df['UTD'] == "GRAÇA")].copy()
gp28 = base_df[(base_df['UTD'] == "PIRAJA")].copy()
gp29 = base_df[(base_df['UTD'] == "PERIPERI")].copy()
gp30 = base_df[(base_df['UTD'] == "ITAPOAN")].copy()
gp31 = base_df[(base_df['UTD'] == "PITUBA")].copy()
gp32 = base_df[(base_df['UTD'] == "CAMAÇARI")].copy()
gp33 = base_df[(base_df['UTD'] == "CANDEIAS")].copy()
gp34 = base_df[(base_df['UTD'] == "LAURO DE FREITAS")].copy()
gp35 = base_df[(base_df['UTD'] == "LIVRAMENTO DE NOSSA SENHORA")].copy()
gp36 = base_df[(base_df['UTD'] == "BRUMADO")].copy()
gp37 = base_df[(base_df['UTD'] == "IRECÊ")].copy()
gp38 = base_df[(base_df['UTD'] == "ITABERABA")].copy()
gp39 = base_df[(base_df['UTD'] == "SEABRA")].copy()
gp40 = base_df[(base_df['UTD'] == "IBOTIRAMA")].copy()
gp41 = base_df[(base_df['UTD'] == "BOM JESUS DA LAPA")].copy()
gp42 = base_df[(base_df['UTD'] == "SANTA MARIA DA VITORIA")].copy()
gp43 = base_df[(base_df['UTD'] == "LUIS EDUARDO MAGALHAES")].copy()
gp44 = base_df[(base_df['UTD'] == "GUANAMBI")].copy()
gp45 = base_df[(base_df['UTD'] == "BARREIRAS")].copy()

In [42]:
base_df[['PACOTE','ZONA']] = base_df[['PACOTE','ZONA']].fillna(0.0)
base_df[['PACOTE','ZONA']]  = base_df[['PACOTE','ZONA']].astype(int)
base_df[['LATITUDE','LONGITUDE']] = base_df[['LATITUDE','LONGITUDE']].astype(float)
centroides = base_df[['LATITUDE','LONGITUDE','NOTA','PACOTE']].copy()
centroides[['LATITUDE','LONGITUDE']] = centroides[['LATITUDE','LONGITUDE']].astype(float)
centroides = centroides.groupby(by =['PACOTE'],as_index= False).mean()

centroides_utd = base_df[['LATITUDE','LONGITUDE','UTD']].copy()
centroides_utd[['LATITUDE','LONGITUDE']] = centroides_utd[['LATITUDE','LONGITUDE']].astype(float)
centroides_utd = centroides_utd.groupby(by =['UTD'],as_index= False).mean()

mp = folium.Map(location=[base_df.LATITUDE.mean(), 
                          base_df.LONGITUDE.mean()], zoom_start=7)
marker_cluster = MarkerCluster(maxClusterRadius=7).add_to(mp)

for i, row in base_df.iterrows():
    atalho_pacote =  row['ATALHO_PACOTE']
    atalho_pacote_2 =  row['ATALHO_PACOTE_2']
    
    if atalho_pacote_2  == '10':
        cor = 'darkgreen'
    elif atalho_pacote_2  == '11':
        cor = 'cadetblue'
    elif atalho_pacote_2  == '12':
        cor = 'darkpurple'
    elif atalho_pacote_2  == '13':
        cor = 'white'
    elif atalho_pacote_2  == '14':
        cor = 'pink'
    elif atalho_pacote_2  == '15':
        cor = 'lightblue'
    elif atalho_pacote_2  == '16':
        cor = 'lightgreen'
    elif atalho_pacote_2  == '17':
        cor = 'gray'
    elif atalho_pacote_2  == '18':
        cor = 'lightblue'
    elif atalho_pacote_2  == '19':
        cor = 'lightgray'
    elif atalho_pacote  == '1':
        cor = 'red'
    elif atalho_pacote  == '2' or atalho_pacote_2  == '24' or atalho_pacote_2  == '33' or atalho_pacote_2  == '45':
        cor = 'blue'
    elif atalho_pacote  == '3' or atalho_pacote_2  == '23' or atalho_pacote_2  == '34' or atalho_pacote_2  == '44':
        cor = 'green'
    elif atalho_pacote  == '4' or atalho_pacote_2  == '25' or atalho_pacote_2  == '35' or atalho_pacote_2  == '43':
        cor = 'purple'
    elif atalho_pacote  == '5' or atalho_pacote_2  == '26' or atalho_pacote_2  == '36' or atalho_pacote_2  == '46':
        cor = 'orange'
    elif atalho_pacote  == '6' or atalho_pacote_2  == '27' or atalho_pacote_2  == '37' or atalho_pacote_2  == '47':
        cor = 'darkred'
    elif atalho_pacote  == '7' or atalho_pacote_2  == '28' or atalho_pacote_2  == '38' or atalho_pacote_2  == '48':
        cor = 'lightred'
    elif atalho_pacote  == '8' or atalho_pacote_2  == '29' or atalho_pacote_2  == '39' or atalho_pacote_2  == '49' or atalho_pacote_2  == '81':
        cor = 'beige'
    elif atalho_pacote  == '9' or atalho_pacote_2  == '23' or atalho_pacote_2  == '33' or atalho_pacote_2  == '43':
        cor = 'darkblue'
    
    utd = row['UTD']
    viatura = row['PACOTE']
    nota = row['NOTA']
    tipo_nota = row['TIPO_NOTA']
    rua= row['ZCGNMLOGR']
    bairro = row['ZCGBAIRR']
    local= row['MUNICIPIO']
    poligonal= row['ZONA']
    legenda = '''

           <b>UTD:</b> {} <br>
           <b>Pacote:</b> {} <br>
           <b>Nota:</b> {} <br>
           <b>Tipo de Nota:</b> {} <br>
           <b>Rua:</b> {} <br>
           <b>Bairro:</b> {} <br>
           <b>Local:</b> {} <br>
           <b>Zona:</b> {} <br>

           '''.format(utd,viatura,nota,tipo_nota,rua,bairro,local,poligonal)
    iframe = folium.IFrame(legenda,width=380, height=230)
    folium.Marker(location = [row['LATITUDE'],row['LONGITUDE']],
                                 popup=folium.Popup(iframe),
                  
                  icon=plugins.BeautifyIcon(icon="arrow-down",icon_shape="marker", number=row['RANK_PECLD'], 
                    border_color= cor, background_color=cor)
                  
                             ).add_to(marker_cluster)
    
    
def get_json_nota(df):
    geo_json = {
      "type": "FeatureCollection",
      "features":[]
    }

    for i,row in df.iterrows():
        temp_dict = {
          "type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates":[row['LONGITUDE'], row['LATITUDE']],

          },"properties": {"nota": row['NOTA']}
        }
        geo_json["features"].append(temp_dict)
    return geo_json

geo_json_markers = get_json_nota(base_df)

geojson_obj = folium.GeoJson(geo_json_markers, show = False, overlay =True).add_to(mp)


def get_json_centroide(df):
    geo_json = {
      "type": "FeatureCollection",
      "features":[]
    }

    for i,row in df.iterrows():
        temp_dict = {
          "type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates":[row['LONGITUDE'], row['LATITUDE']],

          },"properties": {"Viatura": row['PACOTE']}
        }
        geo_json["features"].append(temp_dict)
    return geo_json

geo_json_c_markers = get_json_centroide(centroides)

geojson_obj_c = folium.GeoJson(geo_json_c_markers, show = False, overlay =True).add_to(mp)



def get_json_centroide_utd(df):
    geo_json = {
      "type": "FeatureCollection",
      "features":[]
    }

    for i,row in df.iterrows():
        temp_dict = {
          "type": "Feature",
          "geometry": {
            "type": "Point",
            "coordinates":[row['LONGITUDE'], row['LATITUDE']],

          },"properties": {"UTD": row['UTD']}
        }
        geo_json["features"].append(temp_dict)
    return geo_json

geo_json_utd_markers = get_json_centroide_utd(centroides_utd)

geojson_obj_utd = folium.GeoJson(geo_json_utd_markers, show = False, overlay =True).add_to(mp)



servicesearch = Search(
        layer=geojson_obj,
        search_label='nota',
        placeholder='Procure uma Nota',
        collapsed=False,
        search_zoom = 20
    ).add_to(mp)

servicesearch_centroide = Search(
        layer=geojson_obj_c,
        search_label='Viatura',
        placeholder='Procure um Pacote',
        collapsed=False,
        search_zoom = 14
    ).add_to(mp)

#servicesearch_centroide_utd = Search(
#        layer=geojson_obj_utd,
#        search_label='UTD',
#        placeholder='Procure uma UTD',
#        collapsed=False,
#        search_zoom = 8
#    ).add_to(mp)

folium.LayerControl().add_to(mp)
#mp.save("2teste.html")

minimap = plugins.MiniMap(toggle_display=True)
mp.add_child(minimap)
plugins.Fullscreen(position='topright').add_to(mp)
draw = plugins.Draw(export=False)
draw.add_to(mp)

In [43]:
mp.save('selecao.html')

In [46]:
base_df[['PACOTE','ZONA']] = base_df[['PACOTE','ZONA']].fillna(0.0)
base_df[['PACOTE','ZONA']]  = base_df[['PACOTE','ZONA']].astype(int)

,ZCGNMLOGR,ZCGBAIRR,MUNICIPIO,PACOTE,ZCGACCOUN,LATITUDE,LONGITUDE,UTD,MONTANTE,PECLD,CARTEIRA,TURMA,DATA_PEDIDO,ZONA,NOTA,TIPO_NOTA,RANK_PECLD,ATALHO_PACOTE,ATALHO_PACOTE_2
0,PO REGIAO DE CAFUNDO,RURAL-ITAPITANGA,ITAPITANGA,501.0,007011341467,-14.528522,-39.575907,ITABUNA,3213.89,3213.890000,CORTE,EPS,2022-09-08,185.0,001023413425,CS,1,1,01
1,PO REGIAO DE CAFUNDO,RURAL-ITAPITANGA,ITAPITANGA,501.0,007011341939,-14.528438,-39.575475,ITABUNA,1375.54,1375.540000,CORTE,EPS,2022-09-08,185.0,001023413771,CS,2,1,01
2,AV DO CINQUENTENARIO,CENTRO-ITABUNA,ITABUNA,152.0,000005689210,-14.794237,-39.276618,ITABUNA,7459.93,1363.507814,CORTE,STC,2022-09-08,231.0,001023415153,CS,3,2,52
3,ET SURURU VILA BRASIL,RURAL-BUERAREMA,BUERAREMA,194.0,000202581110,-15.085438,-39.235713,ITABUNA,1910.59,1123.063624,CORTE,STC,2022-09-08,226.0,001023415642,CS,4,4,94
4,RUA A,JARDIM ITALAMAR,ITABUNA,79.0,007021675398,-14.785058,-39.282375,ITABUNA,1160.15,1059.078838,RECORTE,STC,2022-09-08,231.0,002005392470,CA,5,9,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8050,RUA NOVA ESPERANCA,SIM,FEIRA DE SANTANA,17.0,007063620259,-12.256165,-38.913367,FEIRA DE SANTANA SUL,1361.59,NaN,CORTE,STC,2022-09-08,122.0,001023412432,CS,152,7,17
8051,RUA SAO JOAO DO CAZUMBA,CIS,FEIRA DE SANTANA,21.0,007066850239,-12.309622,-38.969150,FEIRA DE SANTANA SUL,751.66,NaN,CORTE,STC,2022-09-08,126.0,001023421089,CS,152,1,21
8052,AV EDUARDO FROES DA MOTA,SAO JOAO,FEIRA DE SANTANA,78.0,000227040319,-12.232558,-38.952697,FEIRA DE SANTANA SUL,1229.72,NaN,CORTE,STC,2022-09-08,123.0,001023411395,CS,152,8,78
8053,RUA TURIM,SANTA MONICA,FEIRA DE SANTANA,78.0,007027042494,-12.267760,-38.934952,FEIRA DE SANTANA SUL,891.53,NaN,CORTE,STC,2022-09-08,124.0,001023418597,CS,152,8,78


,ZCGNMLOGR,ZCGBAIRR,MUNICIPIO,PACOTE,ZCGACCOUN,LATITUDE,LONGITUDE,UTD,MONTANTE,PECLD,CARTEIRA,TURMA,DATA_PEDIDO,ZONA,NOTA,TIPO_NOTA,RANK_PECLD,ATALHO_PACOTE,ATALHO_PACOTE_2
0,PO REGIAO DE CAFUNDO,RURAL-ITAPITANGA,ITAPITANGA,501,007011341467,-14.528522,-39.575907,ITABUNA,3213.89,3213.890000,CORTE,EPS,2022-09-08,185,001023413425,CS,1,1,01
1,PO REGIAO DE CAFUNDO,RURAL-ITAPITANGA,ITAPITANGA,501,007011341939,-14.528438,-39.575475,ITABUNA,1375.54,1375.540000,CORTE,EPS,2022-09-08,185,001023413771,CS,2,1,01
2,AV DO CINQUENTENARIO,CENTRO-ITABUNA,ITABUNA,152,000005689210,-14.794237,-39.276618,ITABUNA,7459.93,1363.507814,CORTE,STC,2022-09-08,231,001023415153,CS,3,2,52
3,ET SURURU VILA BRASIL,RURAL-BUERAREMA,BUERAREMA,194,000202581110,-15.085438,-39.235713,ITABUNA,1910.59,1123.063624,CORTE,STC,2022-09-08,226,001023415642,CS,4,4,94
4,RUA A,JARDIM ITALAMAR,ITABUNA,79,007021675398,-14.785058,-39.282375,ITABUNA,1160.15,1059.078838,RECORTE,STC,2022-09-08,231,002005392470,CA,5,9,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8050,RUA NOVA ESPERANCA,SIM,FEIRA DE SANTANA,17,007063620259,-12.256165,-38.913367,FEIRA DE SANTANA SUL,1361.59,NaN,CORTE,STC,2022-09-08,122,001023412432,CS,152,7,17
8051,RUA SAO JOAO DO CAZUMBA,CIS,FEIRA DE SANTANA,21,007066850239,-12.309622,-38.969150,FEIRA DE SANTANA SUL,751.66,NaN,CORTE,STC,2022-09-08,126,001023421089,CS,152,1,21
8052,AV EDUARDO FROES DA MOTA,SAO JOAO,FEIRA DE SANTANA,78,000227040319,-12.232558,-38.952697,FEIRA DE SANTANA SUL,1229.72,NaN,CORTE,STC,2022-09-08,123,001023411395,CS,152,8,78
8053,RUA TURIM,SANTA MONICA,FEIRA DE SANTANA,78,007027042494,-12.267760,-38.934952,FEIRA DE SANTANA SUL,891.53,NaN,CORTE,STC,2022-09-08,124,001023418597,CS,152,8,78
